Добавлена функция calculate_metrics_with_ci:
- Реализован бутстреп-анализ (1000 итераций)
- Расчет доверительных интервалов для всех метрик (mAP, Precision, Recall)
- Поддержка COCO-формата аннотаций

Особенности реализации:
- Используется бутстреп для оценки неопределенности
- Доверительные интервалы рассчитываются как 2.5% и 97.5% перцентили
- Поддержка всех стандартных метрик детекции

In [ ]:
import os
import torch
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.transforms import functional as F
import torchvision.transforms as T
from PIL import Image
import numpy as np
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_score
from sklearn.utils import resample
import matplotlib.pyplot as plt
from scipy import stats
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

In [34]:
# Конфигурация
# MODEL_PATH = '/home/lastinm/PROJECTS/credit_cards_detection/train/Faster R-CNN/exp/summary/02-05-2025-15-46-48/best_model.pth'
MODEL_PATH = '/home/lastinm/PROJECTS/credit_cards_detection/train/Faster R-CNN/exp/summary/19-05-2025-20-54-34/best_model.pth'
TEST_DATA_DIR = '/home/lastinm/PROJECTS/credit_cards_detection/dataset/coco/test/images'
ANNOTATIONS_FILE = '/home/lastinm/PROJECTS/credit_cards_detection/dataset/coco/test/_annotations.coco.json'  # COCO format
RESULTS_DIR = 'results'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #'cpu' #
NUM_CLASSES = 4  # Фон + ваш класс

In [32]:
class Compose:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        for t in self.transforms:
            image, target = t(image, target)
        return image, target

class ToTensor:
    def __call__(self, image, target):
        return F.to_tensor(image), target

def get_transform():
    return Compose([
        ToTensor(),  # Конвертирует PIL Image в тензор
        # Здесь можно добавить другие преобразования
    ])

In [ ]:
def calculate_metrics_with_ci(predictions, targets, n_bootstrap=1000):
    """Расчет метрик с доверительными интервалами"""
    # Инициализация COCO
    coco_gt = COCO(ANNOTATIONS_FILE)
    
    # Функция для расчета метрик на подвыборке
    def compute_metrics(sub_preds, sub_targets):
        # Преобразование в COCO формат
        coco_results = []
        all_pred_labels = []
        all_true_labels = []
        
        for img_id, (pred, target) in enumerate(zip(sub_preds, sub_targets)):
            # Для Precision
            pred_labels = pred['labels'].cpu().numpy()
            true_labels = [ann['category_id'] for ann in target]
            
            # Для COCO метрик
            for box, score, label in zip(pred['boxes'], pred['scores'], pred['labels']):
                coco_results.append({
                    'image_id': img_id,
                    'category_id': label.item(),
                    'bbox': [box[0].item(), box[1].item(), 
                            (box[2]-box[0]).item(), (box[3]-box[1]).item()],
                    'score': score.item()
                })
            
            # Для Precision
            if len(pred_labels) > 0 and len(true_labels) > 0:
                all_pred_labels.append(pred_labels[0])
                all_true_labels.append(true_labels[0])
        
        # Расчет Precision
        precision = 0.0
        if len(all_pred_labels) > 0 and len(all_true_labels) > 0:
            precision = precision_score(
                all_true_labels,
                all_pred_labels,
                average='macro',
                zero_division=0
            )
        
        # Расчет COCO метрик
        coco_dt = coco_gt.loadRes(coco_results)
        coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
        coco_eval.evaluate()
        coco_eval.accumulate()
        
        return {
            'mAP@0.5:0.95': coco_eval.stats[0],
            'mAP@0.5': coco_eval.stats[1],
            'mAP@0.75': coco_eval.stats[2],
            'Recall@0.5': coco_eval.stats[8],
            'Precision': precision
        }
    
    # Бутстреп-анализ
    bootstrapped_metrics = {
        'mAP@0.5:0.95': [],
        'mAP@0.5': [],
        'mAP@0.75': [],
        'Recall@0.5': [],
        'Precision': []
    }
    
    for _ in range(n_bootstrap):
        # Генерация бутстреп-выборки
        indices = np.random.choice(len(predictions), size=len(predictions), replace=True)
        sub_preds = [predictions[i] for i in indices]
        sub_targets = [targets[i] for i in indices]
        
        # Расчет метрик для выборки
        metrics = compute_metrics(sub_preds, sub_targets)
        for k in bootstrapped_metrics.keys():
            bootstrapped_metrics[k].append(metrics[k])
    
    # Расчет основных метрик
    main_metrics = compute_metrics(predictions, targets)
    
    # Расчет доверительных интервалов
    final_metrics = {}
    for metric_name in bootstrapped_metrics.keys():
        ci_lower = np.percentile(bootstrapped_metrics[metric_name], 2.5)
        ci_upper = np.percentile(bootstrapped_metrics[metric_name], 97.5)
        
        final_metrics[metric_name] = {
            'value': main_metrics[metric_name],
            'ci_lower': ci_lower,
            'ci_upper': ci_upper
        }
    
    return final_metrics

In [35]:
def load_model():
    
    # Загрузка модели с правильной архитектурой
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(weights=None)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, NUM_CLASSES)
    
    # Загрузка весов
    if os.path.exists(MODEL_PATH):
        checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
        
        # Проверка наличия model_state_dict
        if 'model_state_dict' not in checkpoint:
            raise KeyError("Checkpoint does not contain 'model_state_dict'")
        
        # Загрузка весов
        model.load_state_dict(checkpoint['model_state_dict'])
    else:
        raise FileNotFoundError(f"Файл модели не найден: {MODEL_PATH}")
    
    return model.to(DEVICE)

In [37]:
def calculate_metrics(predictions, targets):
    """Расчет mAP и Precision"""
    # Преобразование в COCO формат
    coco_results = []
    all_pred_labels = []
    all_true_labels = []
    
    for img_id, (pred, target) in enumerate(zip(predictions, targets)):
        # Для Precision: собираем предсказанные и истинные классы
        pred_labels = pred['labels'].cpu().numpy()
        true_labels = [ann['category_id'] for ann in target]
        
        # Для COCO метрик
        for box, score, label in zip(pred['boxes'], pred['scores'], pred['labels']):
            coco_results.append({
                'image_id': img_id,
                'category_id': label.item(),
                'bbox': [box[0].item(), box[1].item(), 
                        (box[2]-box[0]).item(), (box[3]-box[1]).item()],
                'score': score.item()
            })
        
        # Для Precision (нужно согласовать количество предсказаний и GT)
        # Здесь простой вариант - сравниваем основные предсказания
        if len(pred_labels) > 0 and len(true_labels) > 0:
            all_pred_labels.append(pred_labels[0])  # Берем самый уверенный bbox
            all_true_labels.append(true_labels[0])   # Берем первый GT bbox
    
    # Расчет Precision
    if len(all_pred_labels) > 0 and len(all_true_labels) > 0:
        precision = precision_score(
            all_true_labels,
            all_pred_labels,
            average='macro',  # Можно использовать 'micro' или 'binary' для одного класса
            zero_division=0
        )
    else:
        precision = 0.0
    
    # Инициализация COCO API
    from pycocotools.coco import COCO
    from pycocotools.cocoeval import COCOeval
    
    coco_gt = COCO(ANNOTATIONS_FILE)
    coco_dt = coco_gt.loadRes(coco_results)
    
    # Оценка COCO
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()
    
    # Сохранение метрик
    metrics = {
        'mAP@0.5:0.95': coco_eval.stats[0],
        'mAP@0.5': coco_eval.stats[1],
        'mAP@0.75': coco_eval.stats[2],
        'Recall@0.5': coco_eval.stats[8],
        'Precision': precision  # Добавляем Precision
    }
    
    with open(os.path.join(RESULTS_DIR, 'metrics.txt'), 'w') as f:
        for k, v in metrics.items():
            f.write(f"{k}: {v:.4f}\n")
            print(f"{k}: {v:.4f}")

Добавлена визуализация:
- График с доверительными интервалами
- Сохранение в файл metrics_with_ci.txt и metrics_with_ci.png

In [ ]:
def plot_metrics_distribution(metrics_with_ci):
    """Визуализация распределения метрик с доверительными интервалами"""
    plt.figure(figsize=(12, 6))
    
    metric_names = list(metrics_with_ci.keys())
    values = [m['value'] for m in metrics_with_ci.values()]
    ci_lower = [m['ci_lower'] for m in metrics_with_ci.values()]
    ci_upper = [m['ci_upper'] for m in metrics_with_ci.values()]
    
    # Ошибки для отображения
    yerr_lower = [v - cl for v, cl in zip(values, ci_lower)]
    yerr_upper = [cu - v for v, cu in zip(values, ci_upper)]
    
    # Построение графика
    plt.bar(metric_names, values, yerr=[yerr_lower, yerr_upper], capsize=5)
    plt.xticks(rotation=45)
    plt.ylabel('Score')
    plt.title('Metrics with 95% Confidence Intervals')
    plt.ylim(0, 1.05)
    plt.grid(True, linestyle='--', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, 'metrics_with_ci.png'), dpi=300)
    plt.close()

In [ ]:
def plot_results(image, prediction, target, idx):
    """Визуализация результатов детекции"""
    # Конвертируем тензор обратно в numpy для отображения
    image = image.cpu().permute(1, 2, 0).numpy()
    image = (image * 255).astype(np.uint8)
    
    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(image)
    
    # Отрисовка GT (Ground Truth)
    for ann in target:  # target - это список аннотаций COCO
        box = ann['bbox']  # COCO использует формат [x,y,width,height]
        # Конвертируем в формат [x1,y1,x2,y2]
        box = [box[0], box[1], box[0]+box[2], box[1]+box[3]]
        rect = plt.Rectangle(
            (box[0], box[1]),
            box[2] - box[0],
            box[3] - box[1],
            fill=False,
            color='green',
            linewidth=2
        )
        ax.add_patch(rect)
    
    # Отрисовка предсказаний
    for box, score in zip(prediction['boxes'], prediction['scores']):
        if score > 0.5:  # Порог уверенности
            box = box.cpu().numpy()  # Конвертируем тензор в numpy
            rect = plt.Rectangle(
                (box[0], box[1]),
                box[2] - box[0],
                box[3] - box[1],
                fill=False,
                color='red',
                linewidth=2
            )
            ax.add_patch(rect)
            ax.text(
                box[0], box[1],
                f'{score:.2f}',
                color='white',
                bbox=dict(facecolor='red', alpha=0.5)
            )
    
    plt.axis('off')
    plt.savefig(
        os.path.join(RESULTS_DIR, f'result_{idx}.png'),
        bbox_inches='tight',
        pad_inches=0
    )
    plt.close()

In [ ]:
def evaluate_faster_rcnn():
    os.makedirs(RESULTS_DIR, exist_ok=True)
    
    # 1. Загрузка модели
    model = load_model()
    model.eval()
    
    # 2. Загрузка тестовых данных
    transform = get_transform()
    dataset = torchvision.datasets.CocoDetection(
        TEST_DATA_DIR,
        ANNOTATIONS_FILE,
        transforms=transform
    )
    
    # 3. Метрики
    all_preds = []
    all_targets = []
    
    for idx in range(len(dataset)):
        image, target = dataset[idx]
        image = image.to(DEVICE)
        
        with torch.no_grad():
            prediction = model([image])[0]
        
        plot_results(image, prediction, target, idx)
        all_preds.append(prediction)
        all_targets.append(target)
    
    # 4. Расчет метрик с доверительными интервалами
    metrics_with_ci = calculate_metrics_with_ci(all_preds, all_targets)
    
    # 5. Сохранение и вывод результатов
    with open(os.path.join(RESULTS_DIR, 'metrics_with_ci.txt'), 'w') as f:
        for metric_name, data in metrics_with_ci.items():
            line = f"{metric_name}: {data['value']:.4f} [{data['ci_lower']:.4f}, {data['ci_upper']:.4f}]"
            print(line)
            f.write(line + '\n')
    
    # Дополнительно: визуализация распределения метрик
    plot_metrics_distribution(metrics_with_ci)

In [28]:
#if __name__ == "__main__":
evaluate_faster_rcnn()

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.02s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.797
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.893
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.883
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.712
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.876
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.821
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | max